In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
file_path = "myData2.parquet"
df = pd.read_parquet(file_path)


In [2]:
df['ML_Node'].value_counts()

ML_Node
0    461500
Name: count, dtype: int64

In [3]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import StandardScaler
from tcn import TCN

2023-12-12 13:18:00.554147: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-12 13:18:00.605889: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-12 13:18:00.605931: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-12 13:18:00.605956: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-12 13:18:00.615141: I tensorflow/core/platform/cpu_feature_g

In [4]:
selected_features = ['timestamp_seconds', # lowers the accuracy 
                     'node_memory_Percpu_bytes', 
                     'node_context_switches_total', 
                     'surfsara_power_usage', 
                     'node_netstat_Tcp_InSegs', 
                     'node_netstat_Tcp_OutSegs', 
                     'node_network_transmit_packets_total-sum', 
                     'node_filesystem_size_bytes-sum', 
                     'node_filesystem_files-sum', 
                     'node_memory_MemFree_bytes', 
                     'node_netstat_Tcp_InErrs']


In [5]:
# Add a new column 'failed_jobs' representing the target variable
df['failed_jobs'] = (df['state'] == 'FAILED').astype(int)

# Extract relevant columns
df_selected = df[['timestamp', 'state'] + selected_features].copy()

# Encode the target variable 'state' to binary (0 for "COMPLETED", 1 otherwise)
df_selected['target'] = (df_selected['state'] != 'COMPLETED').astype(int)

# Drop the original 'state' column
df_selected.drop('state', axis=1, inplace=True)


In [6]:
# Define time intervals
time_intervals = {'minute': '1T', 'hour': '1H', 'day': '1D'}


In [7]:
# Normalize selected features
scaler = MinMaxScaler()
df_selected[selected_features] = scaler.fit_transform(df_selected[selected_features])


In [8]:
# Set sequence length
sequence_length = 30

In [9]:
# Function to prepare data for TCN
def prepare_tcn_data(data, time_interval):
    data.set_index('timestamp', inplace=True) # FixMe
    data_resampled = data.resample(time_interval).sum()
    data_resampled['target'] = data_resampled['target'].clip(upper=1)  # Clip values to 1
    return data_resampled


In [10]:
# Function to create sequences for TCN
def create_tcn_sequences(data, sequence_length):
    sequences, targets = [], []
    for i in range(len(data) - sequence_length):
        seq = data.iloc[i:i+sequence_length].values
        target = data.iloc[i+sequence_length]['target']
        sequences.append(seq)
        targets.append(target)
    return np.array(sequences), np.array(targets)

# Hour

In [11]:
# Prepare data for TCN with minute intervals
tcn_data_hour = prepare_tcn_data(df_selected, time_intervals['hour'])

# Create sequences and targets
sequences_hour, targets_hour = create_tcn_sequences(tcn_data_hour, sequence_length)

# Split the data into training and testing sets
X_train_hour, X_test_hour, y_train_hour, y_test_hour = train_test_split(sequences_hour, targets_hour, test_size=0.3, random_state=42)


In [12]:
# Build the TCN model
tcn_model_hour = Sequential([
    TCN(input_shape=(sequence_length, X_train_hour.shape[2])),
    Dense(1, activation='sigmoid')
])

In [13]:
# Compile the model
tcn_model_hour.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])

# Train the model
tcn_model_hour.fit(X_train_hour, y_train_hour, epochs=20, batch_size=32, validation_split=0.1)

Epoch 1/20
54/54 [==============================] - 5s 47ms/step - loss: 0.4532 - mean_absolute_error: 0.4532 - val_loss: 0.4293 - val_mean_absolute_error: 0.4293
Epoch 2/20
54/54 [==============================] - 2s 38ms/step - loss: 0.4514 - mean_absolute_error: 0.4514 - val_loss: 0.4293 - val_mean_absolute_error: 0.4293
Epoch 3/20
54/54 [==============================] - 2s 39ms/step - loss: 0.4514 - mean_absolute_error: 0.4514 - val_loss: 0.4293 - val_mean_absolute_error: 0.4293
Epoch 4/20
54/54 [==============================] - 2s 37ms/step - loss: 0.4514 - mean_absolute_error: 0.4514 - val_loss: 0.4293 - val_mean_absolute_error: 0.4293
Epoch 5/20
54/54 [==============================] - 2s 36ms/step - loss: 0.4514 - mean_absolute_error: 0.4514 - val_loss: 0.4293 - val_mean_absolute_error: 0.4293
Epoch 6/20
54/54 [==============================] - 2s 34ms/step - loss: 0.4514 - mean_absolute_error: 0.4514 - val_loss: 0.4293 - val_mean_absolute_error: 0.4293
Epoch 7/20
54/54 [====

In [14]:
# Function to make predictions on new data for TCN model
def predict_future_failures_tcn(model, input_data, sequence_length, prediction_steps):
    predictions = []

    for _ in range(prediction_steps):
        # Make a prediction for the next time step
        prediction = model.predict(input_data.reshape(1, sequence_length, input_data.shape[1]))
        predictions.append(prediction[0, 0])

        # Shift the input data by one time step and append the new prediction
        input_data = np.roll(input_data, shift=-1, axis=0)
        input_data[-1, -1] = prediction[0, 0]

    return predictions

In [16]:
# Evaluate the model using Mean Absolute Error
mae_hour = tcn_model_hour.evaluate(X_test_hour, y_test_hour, verbose=0)[1]
print(f'Model Mean Absolute Error: {mae_hour:.4f}')


Model Mean Absolute Error: 0.4347


In [17]:
# Select a starting point for predictions
input_data_tcn = X_test_hour[10]

# Number of time steps to predict into the future
prediction_steps_tcn = 7

# Make predictions with the TCN model
predicted_failures_tcn = predict_future_failures_tcn(tcn_model_hour, input_data_tcn, sequence_length, prediction_steps_tcn)

# Print the predicted failures for TCN
print("Predicted Failures for the Next 7 Time Steps (TCN):")
print(predicted_failures_tcn)

# Evaluate the predictions using Mean Absolute Error
mae_predictions = np.mean(np.abs(predicted_failures_tcn - y_test_hour[10:10+prediction_steps_tcn]))
print(f'Mean Absolute Error for Predictions: {mae_predictions:.4f}')



1/1 [==============================] - 0s 24ms/step


1/1 [==============================] - 0s 26ms/step
Predicted Failures for the Next 7 Time Steps (TCN):
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Mean Absolute Error for Predictions: 0.5714
